In [4]:
# 安装所需spacy库，gensim库
# !pip install spacy
# !pip install gensim

In [5]:
# ! unzip ./data.zip
# ! unzip ./data_utils.zip

In [6]:
# 导入所需文件
import numpy as np
from sklearn.model_selection import ShuffleSplit
from data_utils import ENTITIES, Documents, Dataset, SentenceExtractor, make_predictions
from data_utils import Evaluator
from gensim.models import Word2Vec

In [7]:
# 数据文件读取
data_dir = "./data/train"
ent2idx = dict(zip(ENTITIES, range(1, len(ENTITIES) + 1)))
idx2ent = dict([(v, k) for k, v in ent2idx.items()])

In [8]:
# 训练集，测试集切分与打乱
docs = Documents(data_dir=data_dir)
rs = ShuffleSplit(n_splits=1, test_size=20, random_state=2018)
train_doc_ids, test_doc_ids = next(rs.split(docs))
train_docs, test_docs = docs[train_doc_ids], docs[test_doc_ids]

In [9]:
# 模型参数赋值
num_cates = max(ent2idx.values()) + 1
sent_len = 64
vocab_size = 3000
emb_size = 100
sent_pad = 10
sent_extrator = SentenceExtractor(window_size=sent_len, pad_size=sent_pad)
train_sents = sent_extrator(train_docs)
test_sents = sent_extrator(test_docs)

train_data = Dataset(train_sents, cate2idx=ent2idx)
train_data.build_vocab_dict(vocab_size=vocab_size)

test_data = Dataset(test_sents, word2idx=train_data.word2idx, cate2idx=ent2idx)
vocab_size = len(train_data.word2idx)# 模型参数赋值
num_cates = max(ent2idx.values()) + 1
sent_len = 64
vocab_size = 3000
emb_size = 100
sent_pad = 10
sent_extrator = SentenceExtractor(window_size=sent_len, pad_size=sent_pad)
train_sents = sent_extrator(train_docs)
test_sents = sent_extrator(test_docs)

train_data = Dataset(train_sents, cate2idx=ent2idx)
train_data.build_vocab_dict(vocab_size=vocab_size)

test_data = Dataset(test_sents, word2idx=train_data.word2idx, cate2idx=ent2idx)
vocab_size = len(train_data.word2idx)

In [ ]:
# train_docs[0]

In [35]:
# 词嵌入模型构建
w2v_train_sents = []
for doc in docs:
    w2v_train_sents.append(list(doc.text))
w2v_model = Word2Vec(w2v_train_sents, vector_size=emb_size)

w2v_embeddings = np.zeros((vocab_size, emb_size))
for char, char_idx in train_data.word2idx.items():
    if char in w2v_model.wv:
        w2v_embeddings[char_idx] = w2v_model.wv[char]

In [36]:
w2v_train_sents

[['中',
  '国',
  '成',
  '人',
  '2',
  '型',
  '糖',
  '尿',
  '病',
  'H',
  'B',
  'A',
  '1',
  'C',
  ' ',
  ' ',
  'c',
  '控',
  '制',
  '目',
  '标',
  '的',
  '专',
  '家',
  '共',
  '识',
  '\n',
  '目',
  '前',
  ',',
  '2',
  '型',
  '糖',
  '尿',
  '病',
  '及',
  '其',
  '并',
  '发',
  '症',
  '已',
  '经',
  '成',
  '为',
  '危',
  '害',
  '公',
  '众',
  '\n',
  '健',
  '康',
  '的',
  '主',
  '要',
  '疾',
  '病',
  '之',
  '一',
  ',',
  '控',
  '制',
  '血',
  '糖',
  '是',
  '延',
  '缓',
  '糖',
  '尿',
  '病',
  '进',
  '展',
  '及',
  '\n',
  '其',
  '并',
  '发',
  '症',
  '发',
  '生',
  '的',
  '重',
  '要',
  '措',
  '施',
  '之',
  '一',
  '。',
  '虽',
  '然',
  'H',
  'B',
  'A',
  '1',
  'C',
  ' ',
  ' ',
  '。',
  '是',
  '评',
  '价',
  '血',
  '\n',
  '糖',
  '控',
  '制',
  '水',
  '平',
  '的',
  '公',
  '认',
  '指',
  '标',
  ',',
  '但',
  '应',
  '该',
  '控',
  '制',
  '的',
  '理',
  '想',
  '水',
  '平',
  '即',
  '目',
  '\n',
  '标',
  '值',
  '究',
  '竟',
  '是',
  '多',
  '少',
  '还',
  '存',
  '在',
  '争',
  '议',
  '。',
  '糖',
  '尿',
  '病',
 

In [37]:
# 词嵌入输出
w2v_embeddings[1]# 词嵌入输出
w2v_embeddings[1]

array([-0.08696399, -0.09580569,  0.18961029,  0.110314  , -0.10839769,
       -0.11797536,  0.2000815 , -0.0525223 , -0.02073012, -0.41757369,
        0.24684428, -0.75552654,  0.26167834,  0.16603267, -0.58272535,
        0.16680065,  0.26410547, -0.0574604 , -0.23851469, -0.33694831,
        0.27530986, -0.49797037,  0.33918959, -0.64832443,  0.31764635,
        0.2114438 , -0.57626283, -0.26365876, -0.42124552, -0.13296425,
       -0.00273399,  0.60303748,  0.71550506,  0.39607504,  0.09725308,
       -0.59213334,  0.30065018, -0.62710673, -0.6054337 ,  0.09357762,
       -0.18149757,  0.1432211 ,  0.33051488, -0.08854488,  0.51154447,
        0.10873801,  0.09003051, -0.11415026, -0.49650133, -0.17355533,
       -0.14975588, -0.56320781, -0.01592826,  0.14538863, -0.16820157,
        0.04162791,  0.16834171, -0.13050535,  0.03405711, -0.48854044,
        0.10319384,  0.52973795,  0.07739398, -0.70956922, -0.02437091,
        0.07595708, -0.23162216,  0.09893189, -0.26717764, -0.31